In [1]:
import pandas as pd
import pymysql
import re
import string

In [2]:
ENGINE = pymysql.connect(host="127.0.0.1",
                         user="webserver",
                         passwd="pa55w0rd",
                         db="searchhouse", port=7002)

In [3]:
query = """
       SELECT *
        FROM amenities
"""

df = pd.read_sql(query,ENGINE)
old_list = df['NAME']

In [4]:
full_caps_df = pd.read_csv("amenity_cleanup.csv")
full_caps_list = list(full_caps_df['name'])
# additional
full_caps_list.append("LRT")
full_caps_list.append("A2G")

In [5]:
# return a list of words in string containing substring found in full_caps

def check_in_full_caps(s):
    patts = re.compile("|".join(r"\b{}\b".format(s) for s in full_caps_list), re.I)
    return patts.findall(s)

In [6]:
# returns pos of w in s

def find_index(source_str, word_str): 
    return source_str.index(word_str)

In [7]:
# removes word from source_str at pos

def remove(source_str, pos, word_len):
    source_str = source_str[0:pos:] + source_str[pos+word_len::]
    return source_str

# fullcaps separated by '-'
def remove_all(source_str, word_dict):

    for i in range(len(word_dict)):
        
        word_to_remove = list(word_dict)[i]
        pos = word_dict[word_to_remove][0]
        word_len = word_dict[word_to_remove][1]

        if i > 0:
            
            # decrease pos for next word based on prev word
            prev_word = list(word_dict)[i-1]
            prev_word_len = word_dict[prev_word][1]
            end_pos = pos-prev_word_len
            
            new_str = new_str[0:end_pos:]+new_str[pos::]
        else:
            new_str = source_str[0:pos:] + source_str[pos+word_len::]

    return new_str

In [8]:
# uppercase insert_str, 
# then inserts insert_str at pos of source_str

def insert(source_str, insert_str, pos):

    insert_str = insert_str.upper()
    if pos == 0:
        source_str = insert_str+" "+source_str[pos:]
    else: 
        source_str = source_str[:pos]+insert_str+" "+source_str[pos:]
        
    return source_str

def insert_all(source_str, word_dict):

    source_str = source_str.title()
    
    for i in range(len(word_dict)):
        word_to_insert = list(word_dict)[i]
        pos = word_dict[word_to_insert][0]
        
        # uppercase
        insert_str = word_to_insert.upper()
        if pos == 0:
            source_str = insert_str+" "+source_str[pos:]
        else:
            source_str = source_str[:pos+1]+insert_str+" "+source_str[pos+1:]
    return source_str

In [9]:
def check_busnum(s):
    bus_num = re.findall(r'\.*[0-9]{1,3}[A-Za-z]{1}',s)
    return bus_num

def check_nrbus(s):
    nr_bus = re.findall(r'\.*[A-Za-z]{2}[0-9]{1}',s)
    return nr_bus

def check_word_after_dash(s):
    word_after_dash = re.findall(r'(?<=-).*', s)
    return word_after_dash

def check_word_after_dash_space(s):
    word_after_dash = re.findall(r'(?<=- ).*', s)
    return word_after_dash

def check_word_after_slash(s):
    word_after_slash = re.findall(r'(?<=/).*', s)
    return word_after_slash

def check_word_after_at(s):
    word_after_at = re.findall(r'(?<=@).*', s)
    return word_after_at

def check_mrt_lines(s):
    mrt_lines = re.findall(r'\w*[A-Za-z]{2}[0-9]{1,2}',s)
    return mrt_lines

def check_block_num(s):
    block_num = re.findall(r'\w*[0-9]{1,3}[A-Za-z]{1,2}',s)
    return block_num

def check_word_ends_school(s):
    word_ends_school = re.findall(r'\w*School\b',s)
    return word_ends_school

def check_word_ends_sch(s):
    word_ends_with_sch = re.findall(r'\w*Sch\b',s)
    return word_ends_with_sch

def check_and_in_str(s):
    return " And " in s
    
def check_for_in_str(s):
    return " For " in s

def check_macpherson(s):
    return "Macpherson" in s

def check_of_in_str(s):
    return " Of " in s

In [10]:
def multi_cap_words(source_str, lst):
    # lst contains all cap words in a list
    word_dict = {}

    # remove all
    for each_word in lst:
        word_dict[each_word] = [find_index(source_str, each_word), len(each_word)]

    # exceptional case
    if "St." in source_str and "St" in word_dict.keys():
        del word_dict["St"]

    #hardcoded    
    if "YCK-AMK" in source_str:
        s = "Han Language Centre (YCK-AMK)"

    if "PWD-BCA" in source_str:
        s = "PWD-BCA Clubhouse"

    if not ("St." in source_str and "St" in word_dict.keys()) and not ("YCK-AMK" in source_str) and not ("PWD-BCA Clubhouse" in source_str):

        s = remove_all(source_str, word_dict)

        # insert all at indexes
        s = insert_all(s, word_dict)

        s = remove_space_bracket(s)

        # remove additional double white spaces
        s = re.sub('\s+', ' ', s).strip()
        
    
    return s

In [11]:
def check_spring_ends_school(word, s):
    return word == "Spring" and check_word_ends_school(s)

def check_spring_ends_sch(word, s):
    return word == "Spring" and check_word_ends_sch(s)

def check_mas_ends_school(word, s):
    return word == "Mas" and check_word_ends_school(s)

def check_st_in_word(word):
    return word == "St"


In [12]:
def remove_space_bracket(s):
    s = re.sub(r'\s\)', ')', s)
    return s

def sentence_case(s):
    return string.capwords(s)

# capitalize each word in bracket
def capitalize_bracket_words(s):
    return re.sub(r"\((.*?)\)",lambda x: x.group().title() , s)

In [13]:
def single_cap_words(s, lst):
    # s is the source str, lst contains full cap word
    for each_word in lst:
        # exceptional cases
        if (check_spring_ends_school(each_word, s)) or (check_mas_ends_school(each_word, s)) or (check_spring_ends_sch(each_word, s)) or (check_st_in_word(each_word)):

            s = sentence_case(s)
            
            s = capitalize_bracket_words(s)

            if check_busnum(s) and check_nrbus(s):
                s = re.sub(r'\.*[0-9]{3}[A-Za-z]{1}', lambda x: x.group().upper(), s)
                s = re.sub(r'\.*[A-Za-z]{2}[0-9]{1}', lambda x: x.group().upper(), s)
            if check_nrbus(s):
                s = re.sub(r'\.*[A-Za-z]{2}[0-9]{1}', lambda x: x.group().upper(), s)
            if check_busnum(s):
                s = re.sub(r'\.*[0-9]{3}[A-Za-z]{1}', lambda x: x.group().upper(), s)
                
        else:

            # exception for pa water-venture words
            if "PA Water-Venture" in s:
                pass
            
            else:
                
                word_len = len(each_word)
                index = find_index(s, each_word)

                s = sentence_case(s)
                # cap words in bracket
                s = capitalize_bracket_words(s)
                s = remove(s, index, word_len)
                s = insert(s, each_word, index)

                if check_mrt_lines(s):
                    s = re.sub(r'\w*[A-Za-z]{2}[0-9]{1,2}',lambda x: x.group().upper() , s)
                if check_block_num(s):
                    s = re.sub(r'\w*[0-9]{1,3}[A-Za-z]{1,2}',lambda x: x.group().upper() , s)

                if check_word_after_dash_space(s):
                    s = re.sub(r'(?<=- ).', lambda x: x.group().upper(), s)
                elif check_word_after_dash(s):
                    s = re.sub(r'(?<=-).', lambda x: x.group().upper(), s)

                if check_word_after_at(s):
                    s = re.sub(r'(?<=-).*', lambda x: x.group().upper(), s)
                    
                if check_word_after_slash(s):
                    s = re.sub(r'(?<=/).', lambda x: x.group().upper(), s)
                if check_busnum(s):
                    s = re.sub(r'\.*[0-9]{3}[A-Za-z]{1}', lambda x: x.group().upper(), s)
                if check_nrbus(s):
                    s = re.sub(r'\.*[A-Za-z]{1,2}[0-9]{1,2}', lambda x: x.group().upper(), s)
                if check_busnum(s) and check_nrbus(s):
                    s = re.sub(r'\.*[0-9]{3}[A-Za-z]{1}', lambda x: x.group().upper(), s)
                    s = re.sub(r'\.*[A-Za-z]{2}[0-9]{1}', lambda x: x.group().upper(), s)

                if check_and_in_str(s):
                    s = re.sub(" And ", " and ", s)
  
                # remove additional double white spaces
                s = re.sub('\s+', ' ', s).strip()
                # remove space before brackets
                s = remove_space_bracket(s)
                
                # special amendments
                if "3HOuse Learning Centre (Tampines) [fka MRC Learning Centre (Tampines West)]" in s:
                    s = "3House Learning Centre (Tampines) [fka MRC Learning Centre (Tampines West)]"

        return s

In [37]:
def convert_title_case(s):
    
    if check_in_full_caps(s): # if substring found in full_caps

        find_caps_words = check_in_full_caps(s) # this is a list

        if len(find_caps_words) > 1:
            
            s = multi_cap_words(s, find_caps_words)

        else:

            s = single_cap_words(s, find_caps_words)
     
        
        if check_macpherson(s):
            s = re.sub("Macpherson", "MacPherson", s)
            
        if check_of_in_str(s):
            s = re.sub(" Of ", " of ", s)
        
        if "Sib" in s:
            s = re.sub("Sib", "SIB", s)
        
        if "SP Sma" in s:
            s = re.sub("Sma", "SMA", s)
            
        if "St.gabriel" in s:
            s = re.sub("St.gabriel", "St. Gabriel", s)
              
        if "SPRING Condo" in s:
            s = re.sub("SPRING Condo", "Spring Condo", s)
    
        if "Unisim" in s:
            s = re.sub("Unisim", "UNISIM", s)
            
        return s
    
    else:

        if check_mrt_lines(s):

            s = re.sub(r'\w*[A-Za-z]{2}[0-9]{1,2}',lambda x: x.group().upper() , s)

        if check_block_num(s):
            s = re.sub(r'\w*[0-9]{1,3}[A-Za-z]{1,2}',lambda x: x.group().upper() , s)
        
        if check_word_after_dash_space(s): # w space
            s = re.sub(r'(?<=- ).*', lambda x: string.capwords(x.group()), s)
            # capitalize each word in bracket
            s = capitalize_bracket_words(s)
        
        elif check_word_after_dash(s):  # wo space
            s = re.sub(r'(?<=-).*', lambda x: string.capwords(x.group()), s)
            # capitalize each word in bracket
            s = capitalize_bracket_words(s)

            
        if check_word_after_slash(s):
            s = re.sub(r'(?<=/).', lambda x: x.group().upper(), s)
            
        if check_word_after_at(s):
            s = re.sub(r'(?<=@).', lambda x: x.group().upper(), s)
        if check_busnum(s):
            s = s.title()
            s = re.sub(r'\.*[0-9]{3}[A-Za-z]{1}', lambda x: x.group().upper(), s)
            
        if check_nrbus(s):
            s = re.sub(r'\.*[A-Za-z]{1,2}[0-9]{1,2}', lambda x: x.group().upper(), s)

        if check_nrbus(s) and check_busnum(s):
            s = re.sub(r'\.*[0-9]{3}[A-Za-z]{1}', lambda x: x.group().upper(), s)
            s = re.sub(r'\.*[A-Za-z]{2}[0-9]{1}', lambda x: x.group().upper(), s)

        if not check_mrt_lines(s) and not check_block_num(s) and not check_word_after_dash(s) and not check_busnum(s) and not check_nrbus(s) and not check_word_after_slash(s) and not check_word_after_at(s):
            
            s = string.capwords(s)

            # capitalize each word in bracket
            s = capitalize_bracket_words(s)
        
        # special amendments
        if "3Nc S'Pore (Bus: 254)" in s:
            s = "3NC S'pore (Bus: 254)"
        
        if "Creativebits Centre For New Media Arts" in s:
            s = "CreativeBits Centre For New Media Arts"
            
        if "Jcube" in s:
            s = re.sub("Jcube", "JCube", s)

        if "One Km" in s:
            s = "One KM"
        
        if "Macrichie" in s:
            s = "MacRichie Reservoir"
        
        if check_and_in_str(s):
            s = re.sub(" And ", " and ", s)
    
        if check_for_in_str(s):
            s = re.sub(" For ", " for ", s)
        
        if check_macpherson(s):
            s = re.sub("Macpherson", "MacPherson", s)
            
        if check_of_in_str(s):
            s = re.sub(" Of ", " of ", s)

        if "one-North" in s:
            s = re.sub("one-North", "One-North", s)
            
        if " At " in s:
            s = re.sub(" At ", " at ", s)
            
        if "Ii" in s:
            s = re.sub("Ii", "II", s)
        
        if "R.m.n." in s:
            s = re.sub("R.m.n.", "R.M.N.", s)
        
        if " Emas " in s:
            s = re.sub(" Emas ", " EMAS ", s)
            
        if "Hometeamns" in s:
            s = re.sub("Hometeamns", "HomeTeamNS", s)

        return s

In [40]:
new_list = []
for each in old_list:
    new_list.append(convert_title_case(each))


In [41]:
print (str(new_list))

['One World International School', 'Australian International School', "People's Association HQ", 'Nanyang Academy of Fine Arts (Campus 3)', 'Deutsche Europäische Schule Singapur', 'Hwa Chong International School', 'Lycee Francais De Singapour (French School of Singapore)', 'Norwegian Supplementary School', 'ISS International School (Preston Campus)', 'ISS International School (Elementary & Middle)', 'SJI International School', "People's Association HQ (U/C)", 'Sekolah Indonesia Singapura', 'Waseda Shibuya Senior High School', 'Singapore American School', 'Swiss School', 'United World College of South East Asia (Dover Campus)', 'The Pines Club', 'Nanyang Academy of Fine Arts (Campus 1)', 'Edgefield Primary School', 'Mee Toh School', 'Rivervale Primary School', 'North Spring Primary School', 'Compassvale Primary School', 'Fernvale Primary School', 'Nan Chiau Primary School', 'Seng Kang Primary School', 'Punggol Primary School', "Holy Innocents' Primary School", 'Montfort Junior School', 

In [42]:
# save as csv
new_list_df = pd.DataFrame(new_list)
old_list_df = pd.DataFrame(old_list)
old_list_df['new'] = new_list_df
old_list_df.to_csv("newlist_v2.csv",index=False)

In [43]:
print (len(old_list_df))
old_list_df = old_list_df.rename(columns = {"NAME": "old"})

different_df = old_list_df[old_list_df["old"] != old_list_df["new"]]
different_df.to_csv("difference.csv")

6597


# update table `amenities`

In [ ]:
update_query = """
                UPDATE amenities 
SET 
    NAME = REPLACE(NAME,
        '{}',
        '{}')""".format(old_list,new_list)
#update by id